# XLM-RoBERTa Canonical Training + Phase 3 Export (Colab)

**Model:** XLM-RoBERTa-Base (via HuggingFace Transformers)

**Objective:** Text classification with canonical pipeline alignment:
- Phase 1: Canonical splits (load from Drive) - **SAME AS IMAGE MODELS**
- Phase 2: Canonical classes (27 classes, fp=cdfa70b13f7390e6) - **SAME AS IMAGE MODELS**
- Phase 3: Export contract (.npz + _meta.json) with strict validation

**Expected outputs:**
- `STORE/artifacts/exports/xlmr_canonical/val.npz`
- `STORE/artifacts/exports/xlmr_canonical/val_meta.json`

**Validation:**
- split_signature must match image models: `cf53f8eb169b3531`
- classes_fp must equal canonical: `cdfa70b13f7390e6`
- idx order must align with image models for fusion compatibility

**⚠️ CRITICAL ALIGNMENT:**
- Uses `load_splits()` to load SAME indices as image models
- NO independent train_test_split - ensures text & image use identical samples

In [1]:
from pathlib import Path
import os

from google.colab import drive
drive.mount("/content/drive")

# --- EDIT THESE PATHS ONCE ---
DRIVE_CODE_SNAPSHOT = Path("/content/drive/MyDrive/DS_rakuten_colab")
DRIVE_STORE = Path("/content/drive/MyDrive/DS_rakuten_store")
DRIVE_SPLITS_SRC = DRIVE_STORE / "splits"   # expects train_idx.txt / val_idx.txt / test_idx.txt
# ----------------------------

assert DRIVE_CODE_SNAPSHOT.exists(), f"Missing code snapshot: {DRIVE_CODE_SNAPSHOT}"
DRIVE_STORE.mkdir(parents=True, exist_ok=True)

os.environ["DS_RAKUTEN_STORE"] = str(DRIVE_STORE)

print("✓ DRIVE_CODE_SNAPSHOT:", DRIVE_CODE_SNAPSHOT)
print("✓ DRIVE_STORE:", DRIVE_STORE)
print("✓ DRIVE_SPLITS_SRC:", DRIVE_SPLITS_SRC)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ DRIVE_CODE_SNAPSHOT: /content/drive/MyDrive/DS_rakuten_colab
✓ DRIVE_STORE: /content/drive/MyDrive/DS_rakuten_store
✓ DRIVE_SPLITS_SRC: /content/drive/MyDrive/DS_rakuten_store/splits


In [2]:
import shutil
import sys
from pathlib import Path

RUNTIME_ROOT = Path("/content/DS_rakuten")

# Clean and copy for deterministic imports
if RUNTIME_ROOT.exists():
    shutil.rmtree(RUNTIME_ROOT)

shutil.copytree(DRIVE_CODE_SNAPSHOT, RUNTIME_ROOT)

sys.path.insert(0, str(RUNTIME_ROOT))

print("✓ Runtime code ready:", RUNTIME_ROOT)
print("✓ sys.path[0]:", sys.path[0])

✓ Runtime code ready: /content/DS_rakuten
✓ sys.path[0]: /content/DS_rakuten


In [3]:
from pathlib import Path
import shutil

runtime_splits_dir = Path("/content/DS_rakuten/data/splits")
runtime_splits_dir.mkdir(parents=True, exist_ok=True)

# ⚠️ CRITICAL: Copy SAME split files used by image models
src_files = ["train_idx.txt", "val_idx.txt", "test_idx.txt"]
for fn in src_files:
    src = DRIVE_SPLITS_SRC / fn
    dst = runtime_splits_dir / fn
    assert src.exists(), f"Missing split file in Drive: {src}"
    shutil.copy2(src, dst)
    print(f"✓ Copied {fn}: {src.stat().st_size} bytes")

print("✓ Splits synced to:", runtime_splits_dir)
print("✓ Contents:", list(runtime_splits_dir.glob("*.txt")))

✓ Copied train_idx.txt: 421445 bytes
✓ Copied val_idx.txt: 74328 bytes
✓ Copied test_idx.txt: 87529 bytes
✓ Splits synced to: /content/DS_rakuten/data/splits
✓ Contents: [PosixPath('/content/DS_rakuten/data/splits/test_idx.txt'), PosixPath('/content/DS_rakuten/data/splits/val_idx.txt'), PosixPath('/content/DS_rakuten/data/splits/train_idx.txt')]


In [4]:
# Install dependencies
!pip -q install transformers datasets wandb

# Verify installation
import transformers
import datasets
print(f"✓ transformers version: {transformers.__version__}")
print(f"✓ datasets version: {datasets.__version__}")

✓ transformers version: 4.57.3
✓ datasets version: 4.0.0


In [5]:
# WandB Authentication (REQUIRED for logging!)
import wandb

# Login to WandB
wandb.login()

print("✓ WandB login successful")

wandb: Currently logged in as: xiaosong-dev (xiaosong-dev-formation-data-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✓ WandB login successful


In [6]:
from src.train.text_xlmr import XLMRConfig, run_xlmr_canonical

print("✓ XLMRConfig:", XLMRConfig)
print("✓ run_xlmr_canonical:", run_xlmr_canonical)

✓ XLMRConfig: <class 'src.train.text_xlmr.XLMRConfig'>
✓ run_xlmr_canonical: <function run_xlmr_canonical at 0x7dd7ef242ac0>


In [7]:
# ⚠️ VERIFY SPLIT SIGNATURE (must match image models)
from src.data.split_manager import load_splits, split_signature

splits = load_splits(verbose=True)
sig = split_signature(splits)

print("="*60)
print("SPLIT SIGNATURE VERIFICATION")
print("="*60)
print(f"✓ Signature: {sig}")
print(f"✓ Expected: cf53f8eb169b3531 (from image models)")
print(f"✓ Match: {sig == 'cf53f8eb169b3531'}")
print()
print("Split sizes:")
for k, v in splits.items():
    print(f"  {k}: {len(v)}")
print("="*60)

# CRITICAL: This signature MUST match image models for fusion!

[split_manager] Loading canonical splits from /content/DS_rakuten/data/splits
SPLIT SIGNATURE VERIFICATION
✓ Signature: cf53f8eb169b3531
✓ Expected: cf53f8eb169b3531 (from image models)
✓ Match: True

Split sizes:
  train_idx: 61351
  val_idx: 10827
  test_idx: 12738


In [8]:
import os
from pathlib import Path

STORE = Path(os.environ["DS_RAKUTEN_STORE"])

cfg = XLMRConfig(
    raw_dir=str(STORE / "data_raw"),
    out_dir=str(STORE / "artifacts" / "exports"),
    ckpt_dir=str(STORE / "checkpoints" / "text_xlmr"),

    # Text config
    text_col="designation",
    text_col2="description",
    max_length=384,

    # Training config
    batch_size=128,  # Updated for Colab GPU
    num_epochs=20,
    lr=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,

    # Regularization
    label_smoothing=0.0,
    dropout=0.15,
    patience=2,

    # Model
    model_name="xlm-roberta-base",  # or "FacebookAI/xlm-roberta-large"

    force_colab_loader=True,

    export_name="xlmr_canonical",
    export_split="val",
)

print("="*80)
print("XLM-RoBERTa Training Configuration")
print("="*80)
print(f"Raw dir: {cfg.raw_dir}")
print(f"Export dir: {cfg.out_dir}")
print(f"Checkpoint dir: {cfg.ckpt_dir}")
print(f"Model: {cfg.model_name}")
print(f"Max length: {cfg.max_length}")
print(f"Batch size: {cfg.batch_size}")
print(f"Epochs: {cfg.num_epochs}")
print(f"Export split: {cfg.export_split}")
print("="*80)

XLM-RoBERTa Training Configuration
Raw dir: /content/drive/MyDrive/DS_rakuten_store/data_raw
Export dir: /content/drive/MyDrive/DS_rakuten_store/artifacts/exports
Checkpoint dir: /content/drive/MyDrive/DS_rakuten_store/checkpoints/text_xlmr
Model: xlm-roberta-base
Max length: 384
Batch size: 128
Epochs: 20
Export split: val


In [9]:
# 🚀 RUN TRAINING
result = run_xlmr_canonical(cfg)

print("\n" + "="*80)
print("TRAINING COMPLETE")
print("="*80)
print("\nValidation Metrics:")
for k, v in result["val_metrics"].items():
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")
    else:
        print(f"  {k}: {v}")

print("\nExport Result:")
print(f"  NPZ path: {result['export_result']['npz_path']}")
print(f"  Meta path: {result['export_result']['meta_json_path']}")
print(f"  Probs shape: {result['probs_shape']}")

print("\nVerification:")
print(f"  Model name: {result['verify_metadata']['model_name']}")
print(f"  Split signature: {result['verify_metadata']['split_signature']}")
print(f"  Classes fingerprint: {result['verify_metadata']['classes_fp']}")
print("="*80)

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[INFO] Using Colab data loader (forced via force_colab_loader=True)
[load_data_colab] raw_dir: /content/drive/MyDrive/DS_rakuten_store/data_raw
[load_data_colab] img_root: None
[load_data_colab] X: /content/drive/MyDrive/DS_rakuten_store/data_raw/X_train_update.csv
[load_data_colab] Y: /content/drive/MyDrive/DS_rakuten_store/data_raw/Y_train_CVw08PX.csv
[split_manager] Loading canonical splits from /content/DS_rakuten/data/splits
✓ Split signature: cf53f8eb169b3531
✓ Split sizes: train=61351, val=10827, test=12738
✓ Train size: 61351
✓ Val size: 10827
✓ Test size: 12738
[INFO] Loading tokenizer: xlm-roberta-base


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[INFO] Loading model: xlm-roberta-base
[INFO] Tokenizing datasets...


Map:   0%|          | 0/61351 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,1.615700,1.302555,0.623072,0.484499,0.585562
2,0.811700,0.683996,0.801977,0.751020,0.797321
3,0.586800,0.553249,0.837443,0.798453,0.833995
4,0.473600,0.488303,0.854530,0.828591,0.853521
5,0.428900,0.460596,0.862196,0.840499,0.862049
6,0.370700,0.449622,0.869216,0.852041,0.868915
7,0.328500,0.436576,0.873003,0.854040,0.872370
8,0.289600,0.421914,0.878729,0.862780,0.878209
9,0.252800,0.429036,0.880115,0.865046,0.879828
10,0.234600,0.422020,0.886672,0.871112,0.886435


[INFO] Evaluating on validation set...


✓ Validation metrics: {'eval_loss': 0.42201995849609375, 'eval_accuracy': 0.886672208367969, 'eval_f1_macro': 0.8711121126788469, 'eval_f1_weighted': 0.8864352633634887, 'eval_runtime': 7.405, 'eval_samples_per_second': 1462.127, 'eval_steps_per_second': 5.807, 'epoch': 12.0}
[INFO] Generating predictions for val set...
[OK] Exported model=xlmr_canonical split=val npz=/content/drive/MyDrive/DS_rakuten_store/artifacts/exports/xlmr_canonical/val.npz sig=cf53f8eb169b3531 fp=cdfa70b13f7390e6 n=10827
✓ Export verified: xlmr_canonical
✓ Split signature matches: True
✓ Classes fingerprint matches: True


eval/accuracy,▁▆▇▇▇████████
eval/f1_macro,▁▆▇▇▇████████
eval/f1_weighted,▁▆▇▇▇████████
eval/loss,█▃▂▂▁▁▁▁▁▁▁▁▁
eval/runtime,▆▁▂▁▆▇▂█▅▂▃██
eval/samples_per_second,▃█▇█▃▂▇▁▄▇▆▁▁
eval/steps_per_second,▃█▇█▃▂▇▁▄▇▆▁▁
test/accuracy,▁
test/f1_macro,▁
test/f1_weighted,▁
+9,...



TRAINING COMPLETE

Validation Metrics:
  eval_loss: 0.4220
  eval_accuracy: 0.8867
  eval_f1_macro: 0.8711
  eval_f1_weighted: 0.8864
  eval_runtime: 7.4050
  eval_samples_per_second: 1462.1270
  eval_steps_per_second: 5.8070
  epoch: 12.0000

Export Result:
  NPZ path: /content/drive/MyDrive/DS_rakuten_store/artifacts/exports/xlmr_canonical/val.npz
  Meta path: /content/drive/MyDrive/DS_rakuten_store/artifacts/exports/xlmr_canonical/val_meta.json
  Probs shape: (10827, 27)

Verification:
  Model name: xlmr_canonical
  Split signature: cf53f8eb169b3531
  Classes fingerprint: cdfa70b13f7390e6


In [10]:
# Verify export files exist
import os
from pathlib import Path

STORE = Path(os.environ["DS_RAKUTEN_STORE"])
export_dir = STORE / "artifacts" / "exports" / cfg.export_name

print("Export dir:", export_dir)
print("Contents:", [p.name for p in export_dir.glob("*")])

assert (export_dir / "val.npz").exists(), "Missing val.npz"
assert (export_dir / "val_meta.json").exists(), "Missing val_meta.json"
print("✓ Export files exist.")

Export dir: /content/drive/MyDrive/DS_rakuten_store/artifacts/exports/xlmr_canonical
Contents: ['val.npz', 'val_meta.json']
✓ Export files exist.


In [11]:
# CRITICAL: Verify alignment with image models
import json
from pathlib import Path
import os

STORE = Path(os.environ["DS_RAKUTEN_STORE"])
text_meta = STORE / "artifacts" / "exports" / cfg.export_name / "val_meta.json"

# Compare with image model export (if exists)
image_meta_candidates = [
    STORE / "artifacts" / "exports" / "convnext_canonical_smoke" / "val_meta.json",
    STORE / "artifacts" / "exports" / "swin_canonical" / "val_meta.json",
    STORE / "artifacts" / "exports" / "vit_canonical" / "val_meta.json",
]

text_data = json.loads(text_meta.read_text())
print("="*80)
print("TEXT MODEL METADATA")
print("="*80)
print(f"Model: {text_data['model_name']}")
print(f"Split: {text_data['split_name']}")
print(f"Split signature: {text_data['split_signature']}")
print(f"Classes fingerprint: {text_data['classes_fp']}")
print(f"Num samples: {text_data['num_samples']}")
print(f"Probs shape: {text_data['probs_shape']}")
print()

for img_meta_path in image_meta_candidates:
    if img_meta_path.exists():
        img_data = json.loads(img_meta_path.read_text())
        print("="*80)
        print(f"COMPARISON WITH {img_data['model_name'].upper()}")
        print("="*80)
        print(f"Split signature match: {text_data['split_signature'] == img_data['split_signature']} ✓" if text_data['split_signature'] == img_data['split_signature'] else f"Split signature MISMATCH! ✗")
        print(f"Classes FP match: {text_data['classes_fp'] == img_data['classes_fp']} ✓" if text_data['classes_fp'] == img_data['classes_fp'] else f"Classes FP MISMATCH! ✗")
        print(f"Num samples match: {text_data['num_samples'] == img_data['num_samples']} ✓" if text_data['num_samples'] == img_data['num_samples'] else f"Num samples MISMATCH! ✗")
        print()
        break
else:
    print("⚠️ No image model exports found for comparison")
    print("   Train an image model first to verify alignment")

TEXT MODEL METADATA
Model: xlmr_canonical
Split: val
Split signature: cf53f8eb169b3531
Classes fingerprint: cdfa70b13f7390e6
Num samples: 10827
Probs shape: [10827, 27]

COMPARISON WITH CONVNEXT_CANONICAL_SMOKE
Split signature match: True ✓
Classes FP match: True ✓
Num samples match: True ✓



In [12]:
# Load and inspect predictions
from src.export.model_exporter import load_predictions
from src.data.label_mapping import CANONICAL_CLASSES_FP
from src.data.split_manager import load_splits, split_signature

splits = load_splits(verbose=False)
sig = split_signature(splits)

loaded = load_predictions(
    npz_path=str(result["export_result"]["npz_path"]),
    verify_split_signature=sig,
    verify_classes_fp=CANONICAL_CLASSES_FP,
    require_y_true=True,
)

print("✓ Loaded predictions successfully")
print(f"  Model: {loaded['metadata']['model_name']}")
print(f"  Split: {loaded['metadata']['split_name']}")
print(f"  Signature: {loaded['metadata']['split_signature']}")
print(f"  Fingerprint: {loaded['metadata']['classes_fp']}")
print(f"  Probs shape: {loaded['probs'].shape}")
print(f"  Y_true shape: {loaded['y_true'].shape}")
print(f"  Idx shape: {loaded['idx'].shape}")

✓ Loaded predictions successfully
  Model: xlmr_canonical
  Split: val
  Signature: cf53f8eb169b3531
  Fingerprint: cdfa70b13f7390e6
  Probs shape: (10827, 27)
  Y_true shape: (10827,)
  Idx shape: (10827,)
